In [1]:
# Challenge 1 - ret2win
by ropemporium

Objective: Locate a method that you want to call within the binary.
Call it by overwriting a saved return address on the stack.

There are 4 binaries offered on the site to dig into but I chose the most generic, the x86_64 version.

SyntaxError: invalid syntax (517658613.py, line 2)